In [ ]:
# Introduction:
# This dataset contains information on startup name,industry vertical,location,investors of the startup etc.
# load the dataset and start analysing....

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
import os
print(os.listdir("../input"))                 
star = pd.read_csv("../input/startup_funding.csv") 
star.head()

In [ ]:
star.info()

In [ ]:
# 10 variables and 2372 observations..
# but few columns have a lot of na values...
#so lets start cleaning the dataset and analyse the percentage of na's in the dataset..

In [ ]:
star._get_numeric_data().columns

In [ ]:
star.describe(include=object)

In [ ]:
nas=pd.isnull(star).sum()
nas

In [ ]:
percent_nas = nas/star.shape[0] * 100
percent_nas

In [ ]:
# the subvertical and Amountinusd columns have high %tage of na's 39 and 35 % respectively..

In [ ]:
!pip install missingno

In [ ]:
import missingno
missingno.matrix(star)

In [ ]:
star['AmountInUSD'].head()

In [ ]:
## Converting the String of numbers into float object through string methods and droping na values also..

In [ ]:
star['AmountInUSD']=star['AmountInUSD'].dropna().apply(lambda rstr: float(rstr.replace(',','')))

In [ ]:
star['AmountInUSD'].sum()        ### the Total Startup investment 18.35 us billion....

In [ ]:
star.info()

In [ ]:
star.describe()

In [ ]:
star['AmountInUSD'].plot.box(figsize=(10,5))

In [ ]:
q3 = star['AmountInUSD'].quantile(0.75)
q1 = star['AmountInUSD'].quantile(0.25)
iqr = q3 - q1
iqr

In [ ]:
lw=q1 - 1.5 * iqr
uw=q3 + 1.5 * iqr
print(lw,uw)

In [ ]:
# Possible oultliers above 14445000..its debatable whether it can be considered or not...

#Applying EDA techniques 

# Univariate Analysis....

In [ ]:
#Analysing one column at a time...

In [ ]:
##  Checking for frequency distribution in different industry Verticals...Categorical Column..

In [ ]:
freq=star['IndustryVertical'].value_counts().head(10)
freq

In [ ]:
freq.plot.bar()

In [ ]:
##ploting the frequcncy distribution...AM considering the top 10 verticals only ..
# lots of new startups have come up in Consumer Internet, Technology and Ecommerce Verticals...
##So takeaway: Consumer Internet industry vertical is rocking the startup space...

In [ ]:
## Trying to analyse the LOcation column to check where the investments are pouring in...Location col...

In [ ]:
freq_loc=star['CityLocation'].value_counts().head(10)
freq_loc

In [ ]:
percent=freq_loc/star.shape[0] * 100
percent

In [ ]:
freq_loc.plot.bar()
#percent.plot.bar()

In [ ]:
#The Top 3 investments are happening in Bangalore , Mumbai and New Delhi..with 27% , 19% and 16% respectively..
##So takeaway: Bangalore is the best city for Startup Companies

#Bivariate ANalysis....

In [ ]:
## Basically we are trying to analyse the two columns here...One can be numercial and other can be Categorical col...

In [ ]:
aa=star.boxplot(column='AmountInUSD', by='IndustryVertical',figsize=(30,15), rot=10)
aa.grid(False)

In [ ]:
a=star.boxplot(column='AmountInUSD', by='CityLocation',figsize=(20,10), rot=10)
a.grid(False)

In [ ]:
star.groupby('IndustryVertical')['AmountInUSD'].count().plot.bar(figsize=(20,10))

In [ ]:
star.groupby('CityLocation')['AmountInUSD'].count().plot.bar(figsize=(20,10))

In [ ]:
## The above plots show the amount invested across different verticals 
## The above plot shows the investment across different locations....

In [ ]:
funding_type=star.groupby('InvestmentType')['AmountInUSD'].count()
funding_type

In [ ]:
funding_type.plot.bar()

In [ ]:
## The major funding is happening through Private Equity and Seed Funding...

In [ ]:
p = star.boxplot(column='AmountInUSD', by='InvestmentType', 
               figsize=(20, 10), rot=10)
p.grid(False)

In [ ]:
a=star.groupby('IndustryVertical')['AmountInUSD'].count()
a.sort_values(ascending=False).head()

###  HYPOTHESIS TESTING  ###

In [ ]:
### Applying T Test on one Categorical Column and one numerical coulumn...
# and considering the ConsumerInternet industry vertical and AmountInUSD col..

In [ ]:
##IndustryVertical    VS     Amount Invested...
consumer = star[star['IndustryVertical'] == 'Consumer Internet']['AmountInUSD'].values
technology = star[star['IndustryVertical'] == 'Technology']['AmountInUSD'].values
print(len(consumer), len(technology))

In [ ]:
from scipy.stats import ttest_ind
s, p = ttest_ind(consumer, technology)

if p > 0.05:
    print ('INDUSTRY VERTICAL does not influence Amount Invested')   #Accept the null hypothesis as p >0.05
else:
    print('INDUSTRY VERTICAL does influence Amount Invested')  #Reject the null hypothesis as p <0.05  and accept the alternate hypothesis..        

In [ ]:
### Applying Annova on one Categorical Column and one numerical coulumn...
# and considering the ConsumerInternet industry vertical and AmountInUSD col..

In [ ]:
from scipy.stats import f_oneway
s, p = f_oneway(consumer, technology)

if p > 0.05:
    print ('INDUSTRY VERTICAL does not influence Amount Invested')
else:
    print('INDUSTRY VERTICAL does influence Amount Invested')

In [ ]:
# TAKEAWAY= The investment amount does depend on the Industry Vertical ...
# which has been proved by the above DATAS, PLOTS AND HYPOTHESIS      TESTS ALSO...

## BIVARIATE ANALYSIS....     2 NUMERICAL COLUMNS...

In [ ]:
###  CORELATION  

In [ ]:
star.info()

In [ ]:
star.plot.scatter(x='SNo', y='AmountInUSD')

In [ ]:
star.corr()

In [ ]:
# Since there are no two numerical columns in the dataset we cannot do corelation ...
# as finding corelation between the SNo and Amount invested will not be ideal..

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(13,8)})
p = sns.heatmap(star.corr(), cmap='Blues')

In [ ]:
### not much of inference from the above plot as numerical cols are not present...

## Crosstab Analysis
## Categorical vs Categorical

In [ ]:
obs = star.groupby(['IndustryVertical', 'CityLocation']).size()
obs.name = 'Freq'
obs = obs.reset_index()

obs = obs.pivot_table(index='IndustryVertical', columns='CityLocation',
                values='Freq')
obs
sns.heatmap(obs, cmap='CMRmap_r')

In [ ]:
obs = star.groupby(['IndustryVertical', 'InvestmentType']).size()
obs.name = 'Freq'
obs = obs.reset_index()

obs = obs.pivot_table(index='IndustryVertical', columns='InvestmentType',
                values='Freq')
obs
sns.heatmap(obs, cmap='CMRmap_r')